**Применение apache tvm**


В данном ноутбуке я применил apache tvm для улучшения эффективности работы немного измененной нейросети(чуть чуть "тяжелее"), которую я делал для задания по сверточным нейросетям. Тюнинг модели с помощью autoTVM не был произведен до конца, так как занял бы слишком много времени и неуспел бы наверняка выполниться до заняттия

In [1]:
!pip install keras-flops

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.3 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.10.1 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.61.0 which is incompatible.
tfx-bsl 1.10.1 requires pyarrow<7,>=6, but you have pyarrow 9.0.0 which is incompatible.
tensorflow-transform 1.10.1 requires pyarrow<7,>=6, but you have pyarrow 9.0.0 which is incompatible.
tensorflow-serving-api 2.10.0 requires tensorflow<3,>=2.10.0, but you have tensorflow 2.9.2 which is incompatible.
ortools 9.5.2237 requires protobuf>=4.2

In [2]:

import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [3]:

CLASSES       = 100
BATCH_SIZE    = 32
LEARNING_RATE = 1e-2

In [4]:

from keras.datasets import cifar100
(X_train, y_train), (X_val, y_val) = cifar100.load_data()

169001437/169001437 [==============================] - 11s 0us/step


In [5]:

from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, 100)
y_val = np_utils.to_categorical(y_val, 100)

In [6]:

model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(64, kernel_size=(4,4),padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(32, kernel_size=(3,3), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(16, kernel_size=(2,2), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(300),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(300),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
])

In [7]:

flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

In [8]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        3136      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        18464     
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 32)       1

In [9]:

model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.76)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [10]:
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=7
)

Epoch 1/7
1563/1563 [==============================] - 18s 7ms/step - loss: 3.6379 - accuracy: 0.1486 - val_loss: 3.9330 - val_accuracy: 0.1264
Epoch 2/7
1563/1563 [==============================] - 10s 7ms/step - loss: 2.9731 - accuracy: 0.2570 - val_loss: 2.8724 - val_accuracy: 0.2816
Epoch 3/7
1563/1563 [==============================] - 10s 6ms/step - loss: 2.6069 - accuracy: 0.3318 - val_loss: 2.4026 - val_accuracy: 0.3812
Epoch 4/7
1563/1563 [==============================] - 11s 7ms/step - loss: 2.3499 - accuracy: 0.3846 - val_loss: 2.3607 - val_accuracy: 0.3962
Epoch 5/7
1563/1563 [==============================] - 10s 6ms/step - loss: 2.1706 - accuracy: 0.4213 - val_loss: 2.2323 - val_accuracy: 0.4209
Epoch 6/7
1563/1563 [==============================] - 10s 6ms/step - loss: 2.0484 - accuracy: 0.4490 - val_loss: 2.1682 - val_accuracy: 0.4362
Epoch 7/7
1563/1563 [==============================] - 11s 7ms/step - loss: 1.9633 - accuracy: 0.4661 - val_loss: 2.1325 - val_accuracy:

In [11]:
!pip install apache-tvm

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 11.1 MB/s eta 0:00:0000:0100:01


In [12]:
!pip install --pre apache-tvm~=0.11

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 MB 11.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: apache-tvm
    Found existing installation: apache-tvm 0.10.0
    Uninstalling apache-tvm-0.10.0:
      Successfully uninstalled apache-tvm-0.10.0


In [13]:
import tvm
print(tvm.__version__)

0.12.dev54


In [14]:
from tvm import relay
from tvm.contrib import graph_executor

In [15]:
model.save("cifar100_keras_model")

In [16]:
input_shape = [32, 32, 32, 3] # [batch, height, width, channels]
shape_dict = {"input_1": input_shape}
mod, params = relay.frontend.from_keras(model, shape_dict, layout="NHWC")
print(mod)

def @main(%input_1: Tensor[(32, 32, 32, 3), float32], %v_param_1: Tensor[(4, 4, 3, 64), float32], %v_param_2: Tensor[(64), float32], %v_param_3: Tensor[(64), float32], %v_param_4: Tensor[(64), float32], %v_param_5: Tensor[(64), float32], %v_param_6: Tensor[(64), float32], %v_param_7: Tensor[(3, 3, 64, 32), float32], %v_param_8: Tensor[(32), float32], %v_param_9: Tensor[(32), float32], %v_param_10: Tensor[(32), float32], %v_param_11: Tensor[(32), float32], %v_param_12: Tensor[(32), float32], %v_param_13: Tensor[(2, 2, 32, 16), float32], %v_param_14: Tensor[(16), float32], %v_param_15: Tensor[(16), float32], %v_param_16: Tensor[(16), float32], %v_param_17: Tensor[(16), float32], %v_param_18: Tensor[(16), float32], %v_param_19: Tensor[(300, 1024), float32], %v_param_20: Tensor[(300), float32], %v_param_21: Tensor[(300), float32], %v_param_22: Tensor[(300), float32], %v_param_23: Tensor[(300), float32], %v_param_24: Tensor[(300), float32], %v_param_25: Tensor[(300, 300), float32], %v_param

In [17]:
target = tvm.target.Target("llvm  -mcpu=core-avx2")
dev = tvm.cpu(0)

In [18]:
mod

#[version = "0.0.5"]
def @main(%input_1: Tensor[(32, 32, 32, 3), float32], %v_param_1: Tensor[(4, 4, 3, 64), float32], %v_param_2: Tensor[(64), float32], %v_param_3: Tensor[(64), float32], %v_param_4: Tensor[(64), float32], %v_param_5: Tensor[(64), float32], %v_param_6: Tensor[(64), float32], %v_param_7: Tensor[(3, 3, 64, 32), float32], %v_param_8: Tensor[(32), float32], %v_param_9: Tensor[(32), float32], %v_param_10: Tensor[(32), float32], %v_param_11: Tensor[(32), float32], %v_param_12: Tensor[(32), float32], %v_param_13: Tensor[(2, 2, 32, 16), float32], %v_param_14: Tensor[(16), float32], %v_param_15: Tensor[(16), float32], %v_param_16: Tensor[(16), float32], %v_param_17: Tensor[(16), float32], %v_param_18: Tensor[(16), float32], %v_param_19: Tensor[(300, 1024), float32], %v_param_20: Tensor[(300), float32], %v_param_21: Tensor[(300), float32], %v_param_22: Tensor[(300), float32], %v_param_23: Tensor[(300), float32], %v_param_24: Tensor[(300), float32], %v_param_25: Tensor[(300, 300

In [19]:
dev

cpu(0)

In [20]:
with tvm.transform.PassContext(opt_level=3):
    tvm_model = relay.build_module.create_executor("graph", mod, dev, target, params).evaluate()

In [21]:
!pip install graphviz

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [22]:
import numpy as np

import tvm
from tvm import relay
from tvm.contrib import relay_viz

In [23]:
x = relay.var('x')
v1 = relay.log(x)
v2 = relay.add(v1, v1)
f = relay.Function([x], v2)

In [24]:
print(f)

fn (%x) {
  %0 = log(%x);
  add(%0, %0)
}


In [25]:
relay_mod = tvm.IRModule({"main": f})
viz = relay_viz.RelayVisualizer(relay_mod)
viz.render()

@main([Var(x)])
`--Call 
   |--add 
   |--Call 
   |  |--log 
   |  `--Var(Input) name_hint: x
   `--Call 
      |--log 
      `--Var(Input) name_hint: x


In [26]:
!pip3 install --user tornado psutil 'xgboost<1.6.0' cloudpickle

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.6/173.6 MB 5.2 MB/s eta 0:00:0000:0100:01


In [27]:
import os
model_name = "cifar100_keras_model"
input_name = "input_1"
target = tvm.target.Target("llvm -mcpu=core-avx2")

import multiprocessing
# Set number of threads used for tuning based on the number of
# physical CPU cores on your machine.
num_threads = multiprocessing.cpu_count()
print("Num threads: ", int(num_threads/2))
os.environ["TVM_NUM_THREADS"] = str(int(num_threads/2))

Num threads:  1


In [28]:
def evaluate_performance(lib, data_shape, dtype="float32"):
    # upload parameters to device
    dev = tvm.cpu()
    data_tvm = tvm.nd.array((np.random.uniform(size=data_shape)).astype(dtype))
    module = graph_executor.GraphModule(lib["default"](dev))
    module.set_input(input_name, data_tvm)

    # evaluate
    print("Evaluate inference time cost...")
    print(module.benchmark(dev, number=100, repeat=3))

avtotvm

In [29]:
from tvm import autotvm

from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner

In [30]:
log_file = "/%s.autotvm.log" % model_name

In [31]:
# extract workloads from relay program
def extract_tasks(mod, target, params):
    print("Mod:")
    print(mod)
    print("Extract tasks...")
    tasks = autotvm.task.extract_from_program(
        mod, target=target, params=params
    )
    assert(len(tasks) > 0)
    for idx, task in enumerate(tasks):
        print("Task: %d,  workload: %s" % (idx, task.workload))
    return tasks

In [32]:
tasks = extract_tasks(mod, target, params)

Mod:
def @main(%input_1: Tensor[(32, 32, 32, 3), float32], %v_param_1: Tensor[(4, 4, 3, 64), float32], %v_param_2: Tensor[(64), float32], %v_param_3: Tensor[(64), float32], %v_param_4: Tensor[(64), float32], %v_param_5: Tensor[(64), float32], %v_param_6: Tensor[(64), float32], %v_param_7: Tensor[(3, 3, 64, 32), float32], %v_param_8: Tensor[(32), float32], %v_param_9: Tensor[(32), float32], %v_param_10: Tensor[(32), float32], %v_param_11: Tensor[(32), float32], %v_param_12: Tensor[(32), float32], %v_param_13: Tensor[(2, 2, 32, 16), float32], %v_param_14: Tensor[(16), float32], %v_param_15: Tensor[(16), float32], %v_param_16: Tensor[(16), float32], %v_param_17: Tensor[(16), float32], %v_param_18: Tensor[(16), float32], %v_param_19: Tensor[(300, 1024), float32], %v_param_20: Tensor[(300), float32], %v_param_21: Tensor[(300), float32], %v_param_22: Tensor[(300), float32], %v_param_23: Tensor[(300), float32], %v_param_24: Tensor[(300), float32], %v_param_25: Tensor[(300, 300), float32], %v_

In [33]:
def run_tuning(
    tasks, measure_option, tuner="gridsearch", early_stopping=None, log_filename="tuning.log"
):
    for i, task in enumerate(tasks):
        prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))

        # create tuner
        if tuner == "xgb" or tuner == "xgb-rank":
            tuner_obj = XGBTuner(task, loss_type="rank")
        elif tuner == "ga":
            tuner_obj = GATuner(task, pop_size=50)
        elif tuner == "random":
            tuner_obj = RandomTuner(task)
        elif tuner == "gridsearch":
            tuner_obj = GridSearchTuner(task)
        else:
            raise ValueError("Invalid tuner: " + tuner)

        # do tuning
        n_trial = len(task.config_space)
        tuner_obj.tune(
            n_trial=n_trial,
            early_stopping=early_stopping,
            measure_option=measure_option,
            callbacks=[
                autotvm.callback.progress_bar(n_trial, prefix=prefix),
                autotvm.callback.log_to_file(log_filename),
            ],
        )

In [34]:
measure_option = autotvm.measure_option(
    builder=autotvm.LocalBuilder(),
    runner=autotvm.LocalRunner(repeat=3, enable_cpu_cache_flush=True),
)

In [35]:
# run tuning tasks
run_tuning(tasks, measure_option, tuner="xgb", log_filename=log_file)

[Task  1/ 6]  Current/Best:    1.27/  28.44 GFLOPS | Progress: (62/1188) | 204.81 s

/root/.local/lib/python3.7/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


[Task  2/ 6]  Current/Best:    6.10/  40.33 GFLOPS | Progress: (62/124740) | 164.17 ss

KeyboardInterrupt: 

tooooo long(

In [36]:
def evaluate(module, data_shape, log_file, target="llvm"):
    # compile kernels in default mode
    print("Evaluation of the network compiled in 'default' mode without auto tune:")
    with tvm.transform.PassContext(opt_level=3):
        print("Compile...")
        lib = relay.build(module, target=target, params=params)
        evaluate_performance(lib, data_shape)

    # compile kernels in kernel tuned only mode
    print("\nEvaluation of the network been tuned on kernel level:")
    with autotvm.apply_history_best(log_file):
        print("Compile...")
        with tvm.transform.PassContext(opt_level=3):
            lib = relay.build(module, target=target, params=params)
        evaluate_performance(lib, data_shape)

In [37]:
evaluate(mod, input_shape, log_file, target)

Evaluation of the network compiled in 'default' mode without auto tune:
Compile...
Evaluate inference time cost...
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  209.9550     210.4618     211.6072     207.7960      1.5966   
               

Evaluation of the network been tuned on kernel level:
Compile...
Evaluate inference time cost...
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  203.4743     203.5899     204.2111     202.6218      0.6539   
               


autoscheduler

In [38]:
from tvm import auto_scheduler

In [39]:
log_file = "/%s.auto-scheduler.log" % model_name

In [40]:
# extract workloads from relay program
def extract_tasks(mod, target, params):
    print("Mod:")
    print(mod)
    print("Extract tasks...")
    tasks, task_weights = auto_scheduler.extract_tasks(mod, params, target)
    assert(len(tasks) > 0)

    for idx, task in enumerate(tasks):
        print("Task: %d, desc: %s" % (idx, task.desc))
    return tasks, task_weights

In [41]:
tasks, task_weights = extract_tasks(mod, target, params)

Mod:
def @main(%input_1: Tensor[(32, 32, 32, 3), float32]) {
  %0 = nn.conv2d(%input_1, meta[relay.Constant][0], padding=[1i64, 1i64, 2i64, 2i64], channels=64, kernel_size=[4, 4], data_layout="NHWC", kernel_layout="HWIO");
  %1 = nn.bias_add(%0, meta[relay.Constant][1], axis=-1);
  %2 = nn.batch_norm(%1, meta[relay.Constant][2], meta[relay.Constant][3], meta[relay.Constant][4], meta[relay.Constant][5], axis=3, epsilon=0.001f);
  %3 = %2.0;
  %4 = nn.relu(%3);
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0], layout="NHWC");
  %6 = nn.conv2d(%5, meta[relay.Constant][6], padding=[1i64, 1i64, 1i64, 1i64], channels=32, kernel_size=[3, 3], data_layout="NHWC", kernel_layout="HWIO");
  %7 = nn.bias_add(%6, meta[relay.Constant][7], axis=-1);
  %8 = nn.batch_norm(%7, meta[relay.Constant][8], meta[relay.Constant][9], meta[relay.Constant][10], meta[relay.Constant][11], axis=3, epsilon=0.001f);
  %9 = %8.0;
  %10 = nn.relu(%9);
  %11 = nn.max_pool2d(%10, pool_size=[2

In [42]:
def run_tuning(tasks, task_weights, log_file, n_trials):
    print("Begin tuning...")
    tuner = auto_scheduler.TaskScheduler(tasks, task_weights)
    tune_option = auto_scheduler.TuningOptions(
        num_measure_trials=n_trials,  # change this to 20000 to achieve the best performance
        runner=auto_scheduler.LocalRunner(repeat=10, enable_cpu_cache_flush=True),
        measure_callbacks=[auto_scheduler.RecordToFile(log_file)],
    )

    tuner.tune(tune_option)

In [43]:
run_tuning(tasks, task_weights, log_file,  64*6)

Begin tuning...
----------------------------------------------------------------------
------------------------------  [ Task Scheduler ]
----------------------------------------------------------------------
|  ID  |                       Task Description                        | Latency (ms) | Speed (GFLOPS) | Trials |
-----------------------------------------------------------------------------------------------------------------
|    0 |             vm_mod_fused_nn_conv2d_add_multiply_add_nn_relu_1 |            - |              - |      0 |
|    1 |                                    vm_mod_fused_nn_max_pool2d |            - |              - |      0 |
|    2 |                                  vm_mod_fused_nn_max_pool2d_1 |            - |              - |      0 |
|    3 |                                       vm_mod_fused_nn_softmax |            - |              - |      0 |
|    4 |             vm_mod_fused_nn_conv2d_add_multiply_add_nn_relu_2 |            - |              - |   

In [44]:
def evaluate(module, data_shape, log_file, target="llvm"):
    # compile kernels in default mode
    print("Evaluation of the network compiled in 'default' mode without auto tune:")
    with tvm.transform.PassContext(opt_level=3):
        print("Compile...")
        lib = relay.build(module, target=target, params=params)
        evaluate_performance(lib, data_shape)

    # compile kernels in kernel tuned only mode
    print("\nEvaluation of the network been tuned on kernel level:")
    with auto_scheduler.ApplyHistoryBest(log_file):
        print("Compile...")
        with tvm.transform.PassContext(opt_level=3, config={"relay.backend.use_auto_scheduler": True}):
            lib = relay.build(module, target=target, params=params)
        evaluate_performance(lib, data_shape)

In [45]:
evaluate(mod, input_shape, log_file, target)

Evaluation of the network compiled in 'default' mode without auto tune:
Compile...
Evaluate inference time cost...
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  210.3001     210.4901     210.5982     209.8120      0.3479   
               

Evaluation of the network been tuned on kernel level:
Compile...
Evaluate inference time cost...
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  10.2601      10.2552      10.3258      10.1994       0.0517   
               


meta-scheduler

In [46]:
!pip install pandas

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [47]:
from tvm import meta_schedule as ms

In [48]:
strategy_name = "evolutionary"
work_dir = "{}_meta-scheduler_{}".format(model_name, strategy_name)
target = tvm.target.Target("llvm -mcpu=core-avx2 -num-cores {}".format(int(num_threads/2)))

In [49]:
# extract workloads from relay program
def extract_tasks(mod, target, params, strategy):
    print("Mod:")
    print(mod)
    print("Extract tasks...")
    extracted_tasks = ms.relay_integration.extract_tasks(
        mod, target, params
    )
    assert(len(extracted_tasks) > 0)
    
    tasks, task_weights = ms.relay_integration.extracted_tasks_to_tune_contexts(
        extracted_tasks, work_dir, strategy=strategy
    )

    for idx, task in enumerate(tasks):
        print("Task: %d, desc: %s" % (idx, task.task_name))

    return tasks, task_weights

In [50]:
tasks, task_weights = extract_tasks(mod, target, params, strategy_name)

Mod:
def @main(%input_1: Tensor[(32, 32, 32, 3), float32]) {
  %0 = nn.conv2d(%input_1, meta[relay.Constant][0], padding=[1i64, 1i64, 2i64, 2i64], channels=64, kernel_size=[4, 4], data_layout="NHWC", kernel_layout="HWIO");
  %1 = nn.bias_add(%0, meta[relay.Constant][1], axis=-1);
  %2 = nn.batch_norm(%1, meta[relay.Constant][2], meta[relay.Constant][3], meta[relay.Constant][4], meta[relay.Constant][5], axis=3, epsilon=0.001f);
  %3 = %2.0;
  %4 = nn.relu(%3);
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0], layout="NHWC");
  %6 = nn.conv2d(%5, meta[relay.Constant][6], padding=[1i64, 1i64, 1i64, 1i64], channels=32, kernel_size=[3, 3], data_layout="NHWC", kernel_layout="HWIO");
  %7 = nn.bias_add(%6, meta[relay.Constant][7], axis=-1);
  %8 = nn.batch_norm(%7, meta[relay.Constant][8], meta[relay.Constant][9], meta[relay.Constant][10], meta[relay.Constant][11], axis=3, epsilon=0.001f);
  %9 = %8.0;
  %10 = nn.relu(%9);
  %11 = nn.max_pool2d(%10, pool_size=[2

In [51]:
import os

def run_tuning(tasks, task_weights, work_dir, n_trials):
    if not os.path.exists(work_dir):
        os.mkdir(work_dir)
    print("Begin tuning...")    
    evaluator_config = ms.runner.config.EvaluatorConfig(number=1, repeat=10, enable_cpu_cache_flush=True);
    database = ms.tune.tune_tasks(
        tasks=tasks,
        task_weights=task_weights,
        work_dir=work_dir,
        max_trials_global=n_trials,
        num_trials_per_iter=64,
        max_trials_per_task=256,
        builder=ms.builder.LocalBuilder(),
        runner=ms.runner.LocalRunner(evaluator_config=evaluator_config),
    )

In [52]:
run_tuning(tasks, task_weights, work_dir, 64*9)

2023-02-15 18:07:58 [INFO] [task_scheduler.cc:260] Task #9 has finished. Remaining task(s): 0


,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,fused_nn_conv2d_add_multiply_add_nn_relu,209715200,1,57.9372,3619.6996,3619.6996,128,Y
1,fused_nn_max_pool2d,2097152,1,6.9467,301.8936,301.8936,28,Y
2,fused_nn_conv2d_add_multiply_add_nn_relu_1,303038464,1,75.4028,4018.9281,4018.9281,128,Y
3,fused_nn_max_pool2d_1,262144,1,9.4971,27.6026,27.6026,28,Y
4,fused_nn_conv2d_add_multiply_add_nn_relu_2,8519680,1,43.9406,193.8909,193.8909,64,Y
5,fused_nn_batch_flatten,1,1,0.0003,3.1020,3.1020,1,Y
6,fused_nn_dense_add_add_nn_relu,19689600,1,36.4569,540.0791,540.0791,64,Y
7,fused_nn_dense_add_add_nn_relu_1,5788800,1,30.9628,186.9599,186.9599,64,Y
8,fused_nn_dense_add,1923200,1,32.4844,59.2038,59.2038,64,Y
9,fused_nn_softmax,12800,1,0.7665,16.6988,16.6988,64,Y



Total trials: 633
Total latency (us): 8968.06

2023-02-15 18:07:58 [DEBUG] [task_scheduler.cc:318] 
 ID |                                       Name |      FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
----------------------------------------------------------------------------------------------------------------------------------------------
  0 |   fused_nn_conv2d_add_multiply_add_nn_relu | 209715200 |      1 |        57.9372 |    3619.6996 |             3619.6996 |    128 |    Y 
  1 |                        fused_nn_max_pool2d |   2097152 |      1 |         6.9467 |     301.8936 |              301.8936 |     28 |    Y 
  2 | fused_nn_conv2d_add_multiply_add_nn_relu_1 | 303038464 |      1 |        75.4028 |    4018.9281 |             4018.9281 |    128 |    Y 
  3 |                      fused_nn_max_pool2d_1 |    262144 |      1 |         9.4971 |      27.6026 |               27.6026 |     28 |    Y 
  4 | fused_nn_conv2d_add_multiply_add_nn

In [53]:
def evaluate(module, data_shape, work_dir, target="llvm"):
    # compile kernels in default mode
    print("Evaluation of the network compiled in 'default' mode without auto tune:")
    with tvm.transform.PassContext(opt_level=3):
        print("Compile...")
        lib = relay.build(module, target=target, params=params)
        evaluate_performance(lib, data_shape)

    # compile kernels in kernel tuned only mode
    print("\nEvaluation of the network been tuned on kernel level:")
    print("Compile...")
    database = ms.database.JSONDatabase(f"{work_dir}/database_workload.json",
                                        f"{work_dir}/database_tuning_record.json",
                                        allow_missing=False)
    with tvm.transform.PassContext(opt_level=3):
        lib = ms.relay_integration.compile_relay(database, module, target, params)
        evaluate_performance(lib, data_shape)

In [54]:
evaluate(mod, input_shape, work_dir, target)

Evaluation of the network compiled in 'default' mode without auto tune:
Compile...
Evaluate inference time cost...
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  209.5732     209.9727     210.0590     208.6880      0.6269   
               

Evaluation of the network been tuned on kernel level:
Compile...
Evaluate inference time cost...
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
   9.1653       9.1836       9.2399       9.0725       0.0695   
               
